In [1]:

!pip install sentence-transformers transformers faiss-cpu > /dev/null


from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import faiss
import numpy as np
import torch

hr = """
Leave Policy:
- Annual leave: 18 days
- Sick leave: 12 days
- Maternity leave: 26 weeks
- Paternity leave: 7 days
"""

sop = """
Onboarding:
- Submit ID proof
- Attend HR welcome meeting
- Laptop issued within 3 working days
"""

faq = """
FAQs:
1. How to check leave balance? -> HR Portal > My Leaves
2. How to request laptop? -> IT Helpdesk ticket
"""

documents = [hr, sop, faq]

def chunk(text, size=300):
    return [text[i:i+size] for i in range(0, len(text), size)]

chunks = []
for d in documents:
    chunks.extend(chunk(d))

print("Chunks created:", len(chunks))


model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = model.encode(chunks)
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)


tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
llm = AutoModelForCausalLM.from_pretrained("microsoft/phi-2")
llm.eval()


def answer_question(query):
    q_emb = model.encode([query])

    D, I = index.search(q_emb, 3)
    retrieved = [chunks[i] for i in I[0]]

    context = "\n".join(retrieved)

    prompt = f"""
Use ONLY the following context to answer the question.

Context:
{context}

Question: {query}

Answer:
"""

    inputs = tokenizer(prompt, return_tensors="pt")
    output = llm.generate(**inputs, max_new_tokens=80)
    answer = tokenizer.decode(output[0], skip_special_tokens=True)

    return answer, retrieved

print("System ready. Ask questions!")


Chunks created: 3


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

System ready. Ask questions!


In [2]:
while True:
    q = input("Ask a question (or type 'exit'): ")
    if q.lower() == "exit":
        break

    answer, ctx = answer_question(q)
    print("\nANSWER:\n", answer)
    print("\nCONTEXT USED:")
    for c in ctx:
        print("-", c[:200], "\n")


Ask a question (or type 'exit'): What is the maternity leave duration?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



ANSWER:
 
Use ONLY the following context to answer the question.

Context:

Leave Policy:
- Annual leave: 18 days
- Sick leave: 12 days
- Maternity leave: 26 weeks
- Paternity leave: 7 days


FAQs:
1. How to check leave balance? -> HR Portal > My Leaves
2. How to request laptop? -> IT Helpdesk ticket


Onboarding:
- Submit ID proof
- Attend HR welcome meeting
- Laptop issued within 3 working days


Question: What is the maternity leave duration?

Answer:

The maternity leave duration is 26 weeks.


CONTEXT USED:
- 
Leave Policy:
- Annual leave: 18 days
- Sick leave: 12 days
- Maternity leave: 26 weeks
- Paternity leave: 7 days
 

- 
FAQs:
1. How to check leave balance? -> HR Portal > My Leaves
2. How to request laptop? -> IT Helpdesk ticket
 

- 
Onboarding:
- Submit ID proof
- Attend HR welcome meeting
- Laptop issued within 3 working days
 

Ask a question (or type 'exit'): exit


In [3]:
!pip install gradio

import gradio as gr
def chat(query):
    answer, _ = answer_question(query)
    return answer

gr.Interface(fn=chat, inputs="text", outputs="text", title="InsightHR Demo").launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6c24f5b02a554d0dd3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
